In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from layers import PositionalEmbedding, MultiHeadSelfAttention, ConvLayer
import datetime

2024-07-08 18:27:06.192962: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-08 18:27:06.789236: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.__version__

'2.12.0'

# Load Data

In [6]:
f = np.load("mfcc_fixed.npz")
X, Y = f['X'], f['Y']
fold = 10
X_train =  np.concatenate((X[0:(fold-1)*100], X[fold*100:1000]))
Y_train =  np.concatenate((Y[0:(fold-1)*100], Y[fold*100:1000]))

x_val, y_val= X_train[800:900], Y_train[800:900]
X_train, Y_train = X_train[0:800], Y_train[0:800]

In [13]:
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
kl_divergence = keras.losses.KLDivergence()
lds = lambda x, y: tf.math.reduce_sum(keras.losses.kl_divergence(x, y))
acc_metric = keras.metrics.SparseCategoricalAccuracy()
optimizer = keras.optimizers.Adam()
zeta = 1e-6


def evaluate(eps=4.0, alpha=4.0):
    loss_history = []
    for fold in range(4):
        x_train = np.concatenate((X_train[0:(fold)*200], X_train[(fold+1)*200:800]))
        y_train = np.concatenate((Y_train[0:(fold)*200], Y_train[(fold+1)*200:800]))
        x_test, y_test = X_train[(fold)*200:(fold+1)*200], Y_train[(fold)*200:(fold+1)*200]

        batch_size = 32

        train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
        train_dataset = train_dataset.shuffle(buffer_size=800, reshuffle_each_iteration=True).batch(batch_size, drop_remainder=True)
        val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
        val_dataset = val_dataset.batch(batch_size, drop_remainder=True)
        test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
        test_dataset = test_dataset.batch(batch_size, drop_remainder=True)


        x = x_train[0:batch_size]
        x_rank = tf.rank(x).numpy()
        x_norm_resize_shape = [batch_size] + list(tf.ones(tf.rank(x), dtype=tf.int32).numpy())[1:]

        class CustomModel(keras.Model):

            def train_step(self, data):
                # Unpack the data. Its structure depends on your model and
                # on what you pass to `fit()`.
                x, y = data

                x_p = tf.random.normal(x.shape)
                x_norm = x_p
                for i in range(x_rank-1, 0, -1):
                    x_norm = tf.norm(x_norm, ord=2, axis=int(i))
                x_p /= tf.reshape(x_norm, (batch_size, 1, 1))
                x_p *= zeta

                with tf.GradientTape() as adversarial_tape:
                    adversarial_tape.watch(x_p)
                    y_p = model(x + x_p, training=True)
                    logits = self(x, training=True)
                    l = lds(logits, y_p)
                g = adversarial_tape.gradient(l, x_p)

                g_norm = g
                for i in range(x_rank-1, 0, -1):
                    g_norm = tf.norm(g_norm, ord=2, axis=int(i))

                x_p = eps * g / (tf.reshape(g_norm, x_norm_resize_shape)+1e-6)

                with tf.GradientTape() as model_tape:
                    y_p = self(x + x_p, training=True)
                    logits = self(x, training=True)
                    l = lds(logits, y_p)    # Recalculate regularization
                    loss = self.compute_loss(y=y, y_pred=logits) + alpha * l / batch_size
                grads = model_tape.gradient(loss, self.trainable_weights)
                self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
                for metric in self.metrics:
                    if metric.name == "loss":
                        metric.update_state(loss)
                    else:
                        metric.update_state(y, logits)

                return {m.name: m.result() for m in self.metrics}

        def build_model(d_model_init=64, num_heads=[2, 2], classes=5, input_shape=(137, 15), batch_size=batch_size):
            inputs = keras.layers.Input(shape=input_shape, batch_size=batch_size)
            x = PositionalEmbedding(d_model=d_model_init)(inputs)
        #     for n_heads in num_heads:
        #         x = MultiHeadSelfAttention(d_model=d_model, num_heads=n_heads)(x)
        #         x = ConvLayer(d_model)(x)
            x = MultiHeadSelfAttention(d_model=d_model_init, num_heads=num_heads[0])(x)
            x = ConvLayer(d_model_init)(x)
            x = tf.keras.layers.Permute((2, 1))(x)
            x = PositionalEmbedding(d_model=d_model_init)(x)
            x = MultiHeadSelfAttention(d_model=d_model_init, num_heads=num_heads[0])(x)
            x = ConvLayer(d_model_init)(x)
            x = keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
            x = keras.layers.Dense(classes, activation='softmax')(x)
            return CustomModel(inputs, x)

        model = build_model(input_shape = X[0].shape)

        model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
        mcp_callback = keras.callbacks.ModelCheckpoint("checkpoint.weights.h5",
                                                        monitor="val_loss",
                                                        verbose=1,
                                                        save_best_only=True,
                                                        save_weights_only=True,)
        es_callback = keras.callbacks.EarlyStopping(
                                                    monitor="val_loss",
                                                    min_delta=0,
                                                    patience=100,
                                                    verbose=1,
                                                    mode="auto",
                                                    baseline=None,
                                                    restore_best_weights=True,
                                                    )
        model.fit(train_dataset, validation_data=val_dataset, batch_size=batch_size, epochs=2000,
                callbacks=[es_callback])
        y_hat = model(x_test[0:20], training=True)
        for i in range(1, 19):
            y_hat = np.append(y_hat, model(x_test[20*i:20*i+20], training=False), axis=0)

        loss = 0
        for i in range(15):
            acc_metric.reset_states()
            acc_metric.update_state(y_test, y_hat)
            acc = acc_metric.result().numpy()
            loss += loss_fn(y_test, y_hat).numpy()
        loss = loss / 15
        loss_history.append(loss)
    
    return np.mean(loss_history)
        

In [23]:
evaluate(eps=1., alpha=1.)

Epoch 1/2000


2024-07-08 21:55:13.644370: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [600]
	 [[{{node Placeholder/_1}}]]
2024-07-08 21:55:13.645265: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [600]
	 [[{{node Placeholder/_1}}]]


18/18 [==============================] - ETA: 0s - loss: 1.6097

2024-07-08 21:55:31.888389: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [100]
	 [[{{node Placeholder/_1}}]]


18/18 [==============================] - 20s 130ms/step - loss: 1.6097 - val_loss: 1.6092 - val_accuracy: 0.1979
Epoch 2/2000
18/18 [==============================] - 1s 46ms/step - loss: 1.6089 - val_loss: 1.6054 - val_accuracy: 0.3854
Epoch 3/2000
18/18 [==============================] - 1s 45ms/step - loss: 1.5955 - val_loss: 1.5712 - val_accuracy: 0.5833
Epoch 4/2000
18/18 [==============================] - 1s 50ms/step - loss: 1.5325 - val_loss: 1.4591 - val_accuracy: 0.6875
Epoch 5/2000
18/18 [==============================] - 1s 44ms/step - loss: 1.3897 - val_loss: 1.2751 - val_accuracy: 0.7500
Epoch 6/2000
18/18 [==============================] - 1s 45ms/step - loss: 1.2168 - val_loss: 1.0737 - val_accuracy: 0.8229
Epoch 7/2000
18/18 [==============================] - 1s 46ms/step - loss: 1.0202 - val_loss: 0.8899 - val_accuracy: 0.8750
Epoch 8/2000
18/18 [==============================] - 1s 49ms/step - loss: 0.8020 - val_loss: 0.6839 - val_accuracy: 0.9062
Epoch 9/2000
18/18 

...

2024-07-08 22:09:22.347370: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [600]
	 [[{{node Placeholder/_1}}]]
2024-07-08 22:09:22.348353: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [600]
	 [[{{node Placeholder/_1}}]]


18/18 [==============================] - ETA: 0s - loss: 1.6093

2024-07-08 22:09:40.517207: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [100]
	 [[{{node Placeholder/_1}}]]


18/18 [==============================] - 20s 131ms/step - loss: 1.6093 - val_loss: 1.6073 - val_accuracy: 0.2500
Epoch 2/2000
18/18 [==============================] - 1s 48ms/step - loss: 1.6013 - val_loss: 1.5844 - val_accuracy: 0.5000
Epoch 3/2000
18/18 [==============================] - 1s 49ms/step - loss: 1.5631 - val_loss: 1.5226 - val_accuracy: 0.6562
Epoch 4/2000
18/18 [==============================] - 1s 47ms/step - loss: 1.4628 - val_loss: 1.3762 - val_accuracy: 0.7396
Epoch 5/2000
18/18 [==============================] - 1s 47ms/step - loss: 1.3213 - val_loss: 1.2708 - val_accuracy: 0.7083
Epoch 6/2000
18/18 [==============================] - 1s 48ms/step - loss: 1.2357 - val_loss: 1.1593 - val_accuracy: 0.6979
Epoch 7/2000
18/18 [==============================] - 1s 46ms/step - loss: 1.1276 - val_loss: 1.1759 - val_accuracy: 0.5938
Epoch 8/2000
18/18 [==============================] - 1s 48ms/step - loss: 1.0187 - val_loss: 1.0143 - val_accuracy: 0.6875
Epoch 9/2000
18/18 

0.09535560756921768